In [ ]:
def create_enum_type(self, enum_name: str, values: list):
    """
    Creates an ENUM type in PostgreSQL if it doesn't already exist.
    """
    try:
        # Check if ENUM already exists in pg_type
        self.cur.execute(f"SELECT 1 FROM pg_type WHERE typname = '{enum_name}'")
        if not self.cur.fetchone():
            # Create ENUM type
            values_str = ", ".join(f"'{value}'" for value in values)
            self.cur.execute(f"CREATE TYPE {enum_name} AS ENUM ({values_str})")
            self.conn.commit()
            print(f"ENUM {enum_name} created successfully")
    except Exception as e:
        print(f"Error creating ENUM {enum_name}: {e}")
        self.conn.rollback()

def create_table(self, table_name: str, columns: Dict[str, str], enum_types: Dict[str, list]):
    """
    Create a table with specified columns and ENUM types.
    
    Example:
    create_table(
        "users", 
        {"id": "SERIAL PRIMARY KEY", "status": "status_enum", "role": "role_enum"},
        {"status_enum": ["active", "inactive", "banned"], "role_enum": ["admin", "user", "guest"]}
    )
    """
    try:
        # Step 1: Create ENUM types if they don’t exist
        for enum_name, values in enum_types.items():
            self.create_enum_type(enum_name, values)

        # Step 2: Construct table schema
        columns_str = ", ".join([f"{col} {dtype}" for col, dtype in columns.items()])
        create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_str});"

        # Step 3: Execute the query
        self.cur.execute(create_query)
        self.conn.commit()
        print(f"Table {table_name} created successfully")
    except Exception as e:
        print(f"Error creating table: {e}")
        self.conn.rollback()
